# Lab 8: Simulations

Simulations have many uses, inluding verifying the properties of statistical tests and conducting NHST themselves. You will need to develop this skill, as it nearly wraps up every unit so far.

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **.html file** and submit it in **ELMS**.

In [ ]:
# These lines import the Numpy and Datascience modules.
import numpy as np
import datascience as ds

# These lines do some fancy plotting magic
import matplotlib
# Required to view plots in a notebook
%matplotlib inline
import matplotlib.pyplot as plt
# This is just to make the plots look a certain way
plt.style.use('fivethirtyeight')

## Basketball Data

This exercise uses salary data and game statistics for basketball players from the 2014-2015 NBA season. The data was collected from [Basketball-Reference](http://www.basketball-reference.com) and [Spotrac](http://www.spotrac.com).

Run the cell below to load the player and salary data. 

In [ ]:
player_data = ds.Table().read_table("player_data.csv")
salary_data = ds.Table().read_table("salary_data.csv")
full_data = salary_data.join("PlayerName", player_data, "Name")

# The show method immediately displays the contents of a table. 
# This way, we can display the top of two tables using a single cell.
player_data.show(3)
salary_data.show(3)
full_data.show(3)

Basketball team managers would like to hire players who perform well but don't command high salaries.  From this perspective, a very crude measure of a player's *value* to their team is the number of points the player scored in a season for every **\$1000 of salary** (*Note*: the `Salary` column is in dollars, not thousands of dollars). For example, Al Horford scored 1156 points and has a salary of **\$12 million.** This is equivalent to 12,000 thousands of dollars, so his value is $\frac{1156}{12000}$.

Let's create a table called `full_data_with_value` that's a copy of `full_data`, with an extra column called `"Value"` containing each player's value (according to our crude measure). 

In [ ]:
full_data_with_value = full_data.with_column('Value', full_data.column('Points')/full_data.column('Salary'))
full_data_with_value.show(5)

Then, let's make a histogram of players' values. We specify bins that make the histogram informative, and adjust the range to exclude outliers. Try running the `hist` method without specifying `bins` or `range` to see what you would get otherwise.

In [ ]:
full_data_with_value.hist('Value', bins = 10, range = make_array(0,0.0015))

Now suppose we weren't able to find out every player's salary (perhaps it was too costly to interview each player).  Instead, we have gathered a *simple random sample* of 100 players' salaries.  The cell below loads those data.

In [ ]:
sample_salary_data = ds.Table.read_table("sample_salary_data.csv")
sample_salary_data.show(3)

<font color = 'red'>**Question 1. Make a histogram of the values of the players in `sample_salary_data`, using the same method for measuring value we used above. Use the same bins, too.**</font>

*Hint:* This will take several steps.

In [ ]:
sample_data = player_data.join('Name', sample_salary_data, 'PlayerName')
sample_data_with_value = ...
...

<font color = 'red'>**Question 2. Compare the distributions from `full_data_with_value` and from question 1. How are they different? How are they similar?**</font>


*YOUR ANSWER HERE*

### Statistical inference

*Recall: A parameter is a population value (e.g., average salary among all point guards = 5,000,000 dollars) and a statistic is a sample value (e.g., average salary of subset of point guards = 3,000,000 dollars))*. 

Rather than getting data on every player, imagine that we had gotten data on only a smaller subset of the players. For 492 players, it's not so unreasonable to expect to see all the data, but usually we aren't so lucky. The population of interest is too big or unreachable to collect observations from everyone. Instead, we often make statistical inferences about a large underlying population using a smaller sample.

A *statistical inference* is a statement about some parameter of the underlying population, such as "the average salary of NBA players in 2014 was $3". You may have heard the word "inference" used in other contexts (e.g., logical/deductive inferences). It's important to keep in mind that statistical inferences, unlike, say, logical inferences, can be wrong. In fact, our methods even have built-in amounts of error, such as a Type I error rate (that we set!).

A general strategy for inference using samples is to estimate parameters of the population by computing the same statistics on a sample. This strategy sometimes works well and sometimes doesn't. The degree to which it gives us useful answers depends on several factors—some of which we'll cover today.

To save typing and increase the clarity of your code, we will package the analysis code into a few functions. This will be useful in the rest of this section as we will repeatedly need to create histograms and collect summary statistics from that data.

We've defined the `histograms` function below, which takes a table (entered as 'data') and two column names 'x1' and 'x2'. When we go to use this function, we will set data equal to our table name and put *Age* and *Salary* as the values for the other two arguments. It uses bin widths of 1 year for `Age` and $1,000,000 for `Salary`.

In [ ]:
def histograms(data, col1, col2):
    """
    Draws two histogram for x1 and x2, which have been defaulted to age and salary. This function will only make sense with the player salary data. 
    
    Arguments:
    data, Table: The table with the basketball player data.
    x1 = The first variable of interest. In this case, we will want it to be player age ("Age" in the dataset)
    x2 = The second variable of interest. In this case, we will want it to be player salary ("Salary" in the dataset)
    
    Returns:
    None
    """
    
    # store ages in an array
    x1 = data.column(col1)
    
    # store salaries per millions of dollars in an array
    x2 = data.column(col2)/1000000
    
    # stage data
    data1 = data.drop(col2).with_column(col2, x2)
    
    # find number of bins for age and salary data
    x1_bins = np.arange(min(x1), max(x1) + 2, 1) 
    x2_bins = np.arange(min(x2), max(x2) + 1, 1)
    
    # plot age data
    data1.hist(col1, bins=x1_bins, unit='year')
    plt.title('Age Distribution')
    
    # plot salary data
    data1.hist(col2, bins=x2_bins, unit='Millions of Dollars')
    plt.title('Salary Distribution') 
    
histograms(full_data, "Age", "Salary")
print('Two histograms should be displayed below')

<font color = 'red'>**Question 3. Create a function called `compute_statistics` that takes a table containing ages and salaries and:**</font>
- Draws a histogram of ages
- Draws a histogram of salaries
- Returns a two-element array containing the average age and average salary (in that order)

*Hint:* You can call the `histograms` function to draw the histograms! 

In [ ]:
def compute_statistics(data, col1 = "Age", col2 = "Salary"):
    """
    Draws histograms and computes statistics.
    
    Required libraries: None
    
    Arguments:
    data, Table: The table with the basketball player data.
    col1 = The first variable of interest. In this case, we will want it to be player age ("Age" in the dataset)
    col2 = The second variable of interest. In this case, we will want it to be player salary ("Salary" in the dataset)
    
    Returns:
    A two-element array with average age and average salary.
    """
    
    ...
    age = ...
    salary = ...
    ...
    

full_stats = compute_statistics(full_data)
full_stats

### Simple random sampling

Most times, we won't be able to observe the full population and only be able to look at sample. In a **simple random sample (SRS) without replacement**, we ensure that each player is selected at most once. Imagine writing down each player's name on a card, putting the cards in an box, and shuffling the box.  Then, pull out cards one by one and set them aside, stopping when the specified sample size is reached.

### Producing simple random samples
Sometimes, it’s useful to take random samples even when we have the data for the whole population. It helps us understand sampling accuracy.

### `sample`

The Table method `sample` produces a random sample from the table. By default, it draws at random *with replacement* from the rows of a table. It takes in the sample size as its argument and returns a *table* with only the rows that were selected. The optional argument `with_replacement=False` specifies that the sample should be drawn without replacement.

Run the cell below to see an example call.

In [ ]:
# Just run this cell

salary_data.sample(5, with_replacement=False)

<font color = 'red'>**Question 4. Produce a simple random sample of size 44 from `full_data`. Run your analysis on it again.  Run the cell a few times to see how the histograms and statistics change across different samples.**</font>

- How much does the average age change across samples? 
- What about average salary?

In [ ]:
my_small_srswor_data = ...
my_small_stats = ...
my_small_stats

<font color = 'red'>**Question 5. As in the previous question, analyze several simple random samples of size 100 from `full_data`.**</font>
- Do the histogram shapes seem to change more or less across samples of 100 than across samples of size 44?  
- Are the sample averages and histograms closer to their true values/shape for age or for salary?  What did you expect to see?

In [ ]:
my_large_srswor_data = ...
my_large_stats = ...
my_large_stats

In practice, we won't have access to the full population (`full_data`) like we have here. Instead, we'll only have a sample of the full data. Our goal with inference is to make conclusions about the population using just the sample. 

> Note: This means that in general, you won't be taking samples from the population like we did above. This was just to demonstrate what the sample might look like in relation to the population. We normally don't take a random sample from our data. If you recall, we typically draw a convenience sample. (A problem for a different time and different class.)

## Stents and Strokes

As mentioned before, we usually only have a dataset containing the sample. That is, we usually don't have the full population to sample from. This doesn't mean that the `sample()` method isn't useful. We can still use it to do simulations using a dataset. 

Let's take a look at an example using some data from an experiment. Medical researchers had conjectured that stents could be used to prevent strokes. They actually found that patients who got the stents might be getting strokes more often rather than less often. A NY Times article about this can be found [here](https://www.nytimes.com/2011/09/08/health/research/08stent.html). The data are provided in `stent.csv`. 


In [ ]:
stent = ds.Table.read_table('stent.csv')
stent.show(5)

We can use the `pivot()` method to take a look at the groups and whether they had a stroke. 

In [ ]:
outcome_by_group = stent.pivot('group', 'outcome')
outcome_by_group

Let's calculate the proportion of people who got a stroke in the control group. To do this, we can use the values in the Table above. 

In [ ]:
control_proportion = outcome_by_group.column('control').item(1) / outcome_by_group.column('control').sum()
control_proportion

We can do the same with the treatment group.

In [ ]:
treatment_proportion = outcome_by_group.column('treatment').item(1) / outcome_by_group.column('treatment').sum()
treatment_proportion

<font color = 'red'>**Question 6. Define a function that calculates the difference in probability of a stroke in the treatment group compared to the control group.**</font>
- This function should take three arguments `tbl`, which is a Table that looks like the `stent` Table (containing two variables, group and outcome), and one argument for each variable. 
- The function should return the difference as a number (it will be negative if the control probability is higher).
- Test this function out with the `stent` Table and assign the difference to `observed_difference`. Confirm that it is the same number as calculated above.

In [ ]:
def stroke_proportion_difference(data):
    """
    Required libraries: None
    
    Arguments:
    data, Table: A table that has group and outcome.
    var1: a variable for "group" 
    var2: a varaiable for "outcome"
    
    Return:
    The difference in proportion of people who got a stroke in the treatment group vs the control group.
    """
    
    ...

observed_difference = stroke_proportion_difference(stent)
observed_difference

The `observed_difference` value represents what actually happened in our experiment. However, it's hard to tell whether this number is sufficiently different from 0 to say that there is a relationship. To make that type of conclusion, we will simulate what might have happened if stroke and treatment group were unrelated. We'll use the results from the simulation and try to determine if a difference as big as `observed_difference` could have happened if stroke and treatment group were unrelated.

## Using `sample` to simulate using data

Suppose we wanted to see what the results would have looked like if stents and strokes were actually unrelated. To simulate this, we need to randomly shuffle one of the variables (either group or outcome). Here, we will randomly shuffle the outcome column while keeping the group column the same. This way, we are randomly assigning the outcomes, and resulting proportions would be what we might expect if the two variables were unrelated. 

To do this, we can use `sample()` to get random rows from the `stent` Table equal to the number of rows in the `stent` Table. Since the `sample` method shuffles all of the rows, though, we need to only take the outcome column and combine that with the original group column.  

Take a look at the following code and try to figure out what it does.

In [ ]:
shuffled_outcome = stent.sample(stent.num_rows, with_replacement = False).column('outcome')
simulated_stent = ds.Table().with_columns('group', stent.column('group'),
                                      'outcome', shuffled_outcome)
simulated_stent.show(5)

Now, we might be curious about what difference in stroke proportion might have happened in our simulated data. We can use our function to see what it was in this one simulation. 

In [ ]:
# Make sure you have stroke_proportion_difference defined above!
stroke_proportion_difference(tbl=simulated_stent, var1 = "group", var2 = "outcome")

We can also repeat this many times to get an array of the **differences we might expect if the outcome and group were independent**. That is, we can repeat this simulation many times. The resulting differences in stroke proportion are what we might have expected to see if getting a stroke was unrelated to getting a stent.  

First, we'll define a function that shuffles just the outcome column.
<font color = 'red'>**Question 7. Define a function called `simulated_difference` that takes in a Table that has two columns, group and outcome. The function should shuffle the outcome column, then use that new Table to calculate the difference in proportions between treatment and control groups. Make sure the function returns the simulated difference in proportions.**</font>

*Hint:* You can call `stroke_proportion_difference` that we've defined already. Use that function with the code we've provided for doing one instance of a simulated Table.

In [ ]:
def simulated_difference(data):
    """
    Required libraries: None
    
    Arguments:
    data, Table: A table that has group and outcome.
    
    Return:
    The same Table with the outcome column shuffled.
    """
    
    ...
    
    return ...

Next, we'll loop through 1000 trials, shuffling the outcome column and calculating the difference every time.

<font color = 'red'>**Question 8. Using a `for` loop and the functions we created above, do 1,000 iterations of shuffling the outcome column and calculating the difference between treatment and control groups. Store the differences within `simulated_differences`. The `simulated_differences` object should be an array with 1,000 values.**</font>

In [ ]:
trials = 1000
simulated_differences = ...

for ... in ...:
    ...

Finally, we can look at the `simulated_differences` to see what the distribution looks like.

In [ ]:
ds.Table().with_column('Differences', simulated_differences).hist()

> If the above histogram just has one bar, it's an indication that you did something wrong with the function to calculate the statistic (`stroke_proportion_difference`) or with the function used to simulate the data (`simulated_differences`). Make sure that there's a new (shuffled) dataset being used calculate the difference based on the inputs to the functions.

<font color = 'red'>**Question 9. Do you think that the observed difference (`observed_difference`) in our original data would be considered unusually high in this distribution?**</font>

You can see what proportion of simulations had a different just as high or higher using the following code:

In [ ]:
np.count_nonzero(simulated_differences >= stroke_proportion_difference(tbl=stent, var1 = "group", var2 = "outcome")) / trials